<a href="https://colab.research.google.com/github/mikoesnim0/DigitalCPATH-research/blob/main/ImgeTile512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#presetting
#is_csv
is_csv = True

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!git clone https://github.com/ncoudray/DeepPATH.git

Cloning into 'DeepPATH'...
remote: Enumerating objects: 2977, done.
remote: Counting objects: 100% (494/494), done.
remote: Compressing objects: 100% (207/207), done.
remote: Total 2977 (delta 297), reused 473 (delta 286), pack-reused 2483
Receiving objects: 100% (2977/2977), 11.03 MiB | 10.76 MiB/s, done.
Resolving deltas: 100% (1716/1716), done.


In [ ]:
%cd "/content/DeepPATH/DeepPATH_code"

/content/DeepPATH/DeepPATH_code


In [ ]:
#importing openslide,
# !pip install openslide
!pip install openslide-python  # For handling slide images
#3rd 시도
!apt-get install openslide-tools  # Install Openslide
!pip install openslide-python    # Install Python bindings for Openslide
!pip install pydicom

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 359.0/359.0 kB 6.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for openslide-python: filename=openslide_python-1.3.1-cp310-cp310-linux_x86_64.whl size=33554 sha256=16eb539dfb68d288c3a16089782a039441547abd98b1286a612a418dfa16bafd
  Stored in directory: /root/.cache/pip/wheels/79/79/fa/29a0087493c69dff7fd0b70fab5d6771002a531010161d2d97
Successfully built openslide-python
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libopenslide0
Suggested packages:
  libtiff-tools
The following NEW packages will be installed:
  libopenslide0 openslide-tools
0 upgraded, 2 newly installed, 0 to remove and 39 not upgraded.
Need to get 104 kB of archives.
After this operation, 297 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libopenslide0 amd64 3.4.1+dfsg-5build1 [89.8 kB]

In [ ]:
#연동되는 xml 파일 확인
import re
import glob
# Define the paths
svs_path = "/content/drive/MyDrive/ImgFiles"
dzi_path = "/content/drive/MyDrive/ImgFiles/Tile"
# Get the list of SVS and XML files
svs_files = glob.glob(svs_path + "/*.svs")
dzi_files = glob.glob(dzi_path + "/*.dzi")

#Start from here if uploaded new files

In [ ]:
import pandas as pd
import time
csv_path = svs_path + '/logue.csv'
if not is_csv:
  data = {
      'ID': [],
      'STime': [],
      'TTime': [],
      'Time':[],
      'Succeed':[]
  }
  df = pd.DataFrame(data)
  df.to_csv(csv_path)
else:
  df = pd.read_csv(csv_path)
df.head()

,Unnamed: 0,ID,STime,TTime,Time,Succeed
0,0,1028059,2024-03-26 09:05:08.413045,2024-03-26 09:21:51.736256,0 days 00:16:43.323211,True
1,1,1028058,2024-03-26 09:21:51.780901,2024-03-26 09:49:17.626592,0 days 00:27:25.845691,True
2,2,1028060,2024-03-26 09:49:17.680864,2024-03-26 10:24:04.616897,0 days 00:34:46.936033,True


In [ ]:
# Function to extract file numbers from file paths
def extract_svs(file_path):
    # Use regular expression to extract the file number
    match = re.search(r'(\d+)\.(svs)$', file_path)
    if match:
        return match.group(1)
    else:
        return None

def extract_dzi(file_path):
    # Use regular expression to extract the file number
    match = re.search(r'(\d+)\.(dzi)$', file_path)
    if match:
        return match.group(1)
    else:
        return None

# Extract file numbers from SVS and XML files
svs_numbers = [extract_svs(file) for file in svs_files]
finis_numbers = list(df['ID'])
dzi_numbers = [extract_dzi(file) for file in dzi_files]

svs_numbers = [number for number in svs_numbers if (number not in finis_numbers and int(number) not in finis_numbers)]

In [ ]:
svs_numbers

['1028118', '1028120']

In [ ]:
# Iterate over files in batches
import os
try:
  fileN = svs_numbers[0]
  svs_file_path = os.path.join(svs_path,fileN + '.svs')

  print(svs_file_path)
except Exception as e:
  print(f"and error occured as {e}")

/content/drive/MyDrive/ImgFiles/1028059.svs


In [ ]:
# !python 00_preprocessing/0b_tileLoop_deepzoom6.py -s 512 -e 0 -j 2 -B 50 -M 20 -x "/content/drive/MyDrive/ImgFiles" -o "/content/drive/MyDrive/ImgFiles/Tile" "{svs_file_path}"

In [ ]:
#svs 파일 업로드되거나 일단 타일링 또는 진행중인 파일이 무엇인지 표시되도록 하는 기능 추가 필요.
import time
for fileN in svs_numbers:
  svs_file_path = svs_path +'/'+ str(fileN)+'.svs'
  print(svs_file_path)
  now = pd.Timestamp.now()
  success = True

  tile_pth = '/content/drive/MyDrive/ImgFiles/Tile'

  try:
    !python 00_preprocessing/0b_tileLoop_deepzoom6.py -s 512 -e 0 -j 2 -B 50 -M 20 -x "/content/drive/MyDrive/ImgFiles" -o "/content/drive/MyDrive/ImgFiles/Tile" "{svs_file_path}"
  except Exception as e:
    success = False
    print(e)
  end = pd.Timestamp.now()
  df.loc[len(df.index)] = [fileN, fileN, now, end ,end-now, success ]  # adding a row
  df.to_csv(csv_path)

/content/drive/MyDrive/ImgFiles/1028118.svs
{'limit_bounds': True, 'overlap': 0, 'format': 'jpeg', 'workers': 2, 'basename': '/content/drive/MyDrive/ImgFiles/Tile', 'quality': 90, 'with_viewer': None, 'tile_size': 512, 'Bkg': 50.0, 'Std': 0, 'xmlfile': '/content/drive/MyDrive/ImgFiles', 'Fieldxml': 'Value', 'mask_type': 1, 'ROIpc': 50, 'oLabelref': '', 'SaveMasks': False, 'tmp_dcm': None, 'srh': 0, 'Mag': 20.0, 'pixelsize': -1, 'pixelsizerange': 0, 'normalize': None}
['/content/drive/MyDrive/ImgFiles/1028118.svs']
/content/drive/MyDrive/ImgFiles/1028118.svs
list of files:
['/content/drive/MyDrive/ImgFiles/1028118.svs']
***********************
processing: 1028118 with extension: svs
worker 2 started with 512 , 0
worker 2 started with 512 , 0
tiler init
start tiler
wsize and overlap:  512   0
wsize and overlap:  512   0
image size: 123503.0 ,  88570.0
Img_Fact:
8.233533333333334
111 :20.0 and pixelsize:-1
range(17, -1, -1)
PercentMasked_0 0.0000
PercentMasked_0 0.0000
PercentMasked_0 0.0